In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
import os
import warnings

# --- Importar función de normalización ---
try:
    from normalization_functions import pipeline_b_normalize
    print("Función 'pipeline_b_normalize' importada correctamente.")
except ImportError:
    print("\n¡Error de Importación!")
    print("Asegúrate de que '1_normalization_pipelines.py' esté en la misma carpeta 'src/'.")
    exit()


Recurso 'stopwords' de NLTK ya está descargado.
Cargadas 313 stopwords en español.
Modelo 'es_core_news_sm' de spaCy cargado.
Función 'pipeline_b_normalize' importada correctamente.


In [3]:

# --- 1. Configuración de Rutas ---
TRAIN_PATH = os.path.join("..", "data", "processed", "train.csv")
RESULTS_PATH = os.path.join("..", "results", "fase_1_pipeline_B.csv")

# Asegurarse que la carpeta de resultados exista
os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)

# Ignorar warnings de convergencia de LogisticRegression para una salida limpia
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# --- 2. Definir "Ingredientes" del Experimento ---

# Las 3 representaciones de texto
vectorizers = {
    "Binary": CountVectorizer(binary=True),
    "Frequency": CountVectorizer(binary=False),
    "TF-IDF": TfidfVectorizer()
}

# Los 2 modelos simples
models = {
    "MultinomialNB": MultinomialNB(),
    # Aumentar max_iter para que LogisticRegression converja con estos datos
    "LogisticRegression": LogisticRegression(max_iter=2000, random_state=0)
}

# --- 3. Cargar y Preparar Datos ---
print(f"Iniciando Experimento: Pipeline B")
try:
    df_train = pd.read_csv(TRAIN_PATH)
    print(f"Datos de entrenamiento cargados desde '{TRAIN_PATH}'.")
except FileNotFoundError:
    print(f"\n¡Error de Archivo!")
    print(f"No se encontró '{TRAIN_PATH}'.")
    print("Asegúrate de haber corrido '0_split_data.py' primero.")
    exit()

print("Aplicando 'pipeline_b_normalize' a los datos de entrenamiento...")
# Aplicar la normalización B a toda la columna 'text'
X_train = df_train['text'].apply(pipeline_b_normalize)
y_train = df_train['Polarity']
print(f"Datos normalizados. {len(X_train)} documentos listos.")


Iniciando Experimento: Pipeline B
Datos de entrenamiento cargados desde '..\data\processed\train.csv'.
Aplicando 'pipeline_b_normalize' a los datos de entrenamiento...
Datos normalizados. 24169 documentos listos.


In [4]:

# --- 4. Bucle de Experimentación (Cross-Validation) ---
print("\nIniciando bucle de validación cruzada (CV=5)...")

experiment_results = []

# Iterar sobre las 3 representaciones
for vec_name, vectorizer in vectorizers.items():
    
    # Iterar sobre los 2 modelos
    for model_name, model in models.items():
        
        # Crear un Pipeline de scikit-learn
        # Esto junta la vectorización y la clasificación en un solo objeto
        # Es la forma *correcta* de usar cross_validate,
        # ya que asegura que la vectorización se aprenda en cada fold.
        text_pipeline = Pipeline([
            ('vectorizer', vectorizer), # Paso 1: Convertir texto a números
            ('classifier', model)       # Paso 2: Clasificar
        ])
        
        print(f"-> Ejecutando: (Pipeline B) + ({vec_name}) + ({model_name})")
        
        # Ejecutar la validación cruzada de 5 folds 
        # y calcular el f1_macro 
        cv_scores = cross_validate(
            estimator=text_pipeline,
            X=X_train, # Los textos ya normalizados
            y=y_train,
            cv=5,               
            scoring='f1_macro', 
            n_jobs=-1           # Usar todos los núcleos de CPU
        )
        
        # Calcular el promedio y desviación de los 5 scores
        avg_f1 = cv_scores['test_score'].mean()
        std_f1 = cv_scores['test_score'].std()
        
        print(f"  -> F1-Macro Promedio: {avg_f1:.4f} (std: {std_f1:.4f})")
        
        # Guardar el resultado
        experiment_results.append({
            "pipeline": "B (A + Stopwords)",
            "vectorizer": vec_name,
            "model": model_name,
            "avg_f1_macro": avg_f1,
            "std_f1_macro": std_f1
        })

# --- 5. Guardar y Reportar Resultados ---
results_df = pd.DataFrame(experiment_results)

# Guardar tu trabajo en un CSV en la carpeta de resultados
try:
    results_df.to_csv(RESULTS_PATH, index=False)
except Exception as e:
    print(f"\n¡Error al guardar resultados! {e}")
    
print("\n--- Resultados Finales (Pipeline B) ---")
# Imprimir los resultados ordenados por el mejor F1
print(results_df.sort_values(by="avg_f1_macro", ascending=False).to_string())
print(f"\n¡Experimento completo! Resultados guardados en '{RESULTS_PATH}'")


Iniciando bucle de validación cruzada (CV=5)...
-> Ejecutando: (Pipeline B) + (Binary) + (MultinomialNB)
  -> F1-Macro Promedio: 0.2859 (std: 0.0082)
-> Ejecutando: (Pipeline B) + (Binary) + (LogisticRegression)
  -> F1-Macro Promedio: 0.4459 (std: 0.0129)
-> Ejecutando: (Pipeline B) + (Frequency) + (MultinomialNB)
  -> F1-Macro Promedio: 0.3160 (std: 0.0074)
-> Ejecutando: (Pipeline B) + (Frequency) + (LogisticRegression)
  -> F1-Macro Promedio: 0.4486 (std: 0.0138)
-> Ejecutando: (Pipeline B) + (TF-IDF) + (MultinomialNB)
  -> F1-Macro Promedio: 0.1639 (std: 0.0004)
-> Ejecutando: (Pipeline B) + (TF-IDF) + (LogisticRegression)
  -> F1-Macro Promedio: 0.3676 (std: 0.0080)

--- Resultados Finales (Pipeline B) ---
            pipeline vectorizer               model  avg_f1_macro  std_f1_macro
3  B (A + Stopwords)  Frequency  LogisticRegression      0.448626      0.013803
1  B (A + Stopwords)     Binary  LogisticRegression      0.445851      0.012893
5  B (A + Stopwords)     TF-IDF  Logi